In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Input data

In [2]:
#Import delay delivery datasets
#Import delay delivery datasets
data_delay_4_6 = pd.read_csv(r"D:\Code\Python\TH_Tien_Xu_Ly\ds-108-o-21-assigment-06\delay_4_6_CONDITION_PRODUCT_SUPPLIER.csv", encoding='utf-8')
data_delay_7_9 = pd.read_csv(r"D:\Code\Python\TH_Tien_Xu_Ly\ds-108-o-21-assigment-06\delay_7_9_CONDITION_PRODUCT_SUPPLIER.csv", encoding='utf-8')

#Import not delay delivery datasets
data_not_delay_4_6 = pd.read_csv(r"D:\Code\Python\TH_Tien_Xu_Ly\ds-108-o-21-assigment-06\not_delay_4_6_CONDITION_PRODUCT_SUPPLIER.csv", encoding='utf-8')
data_not_delay_7_9 = pd.read_csv(r"D:\Code\Python\TH_Tien_Xu_Ly\ds-108-o-21-assigment-06\not_delay_7_9_CONDITION_PRODUCT_SUPPLIER.csv", encoding='utf-8')
# Import test data
test = pd.read_csv(r"D:\Code\Python\TH_Tien_Xu_Ly\ds-108-o-21-assigment-06\PILOT_10.csv", encoding='utf-8')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_31352\1049567294.py:7: DtypeWarning: Columns (10,42) have mixed types. Specify dtype option on import or set low_memory=False.
  data_not_delay_4_6 = pd.read_csv(r"D:\Code\Python\TH_Tien_Xu_Ly\ds-108-o-21-assigment-06\not_delay_4_6_CONDITION_PRODUCT_SUPPLIER.csv", encoding='utf-8')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_31352\1049567294.py:8: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  data_not_delay_7_9 = pd.read_csv(r"D:\Code\Python\TH_Tien_Xu_Ly\ds-108-o-21-assigment-06\not_delay_7_9_CONDITION_PRODUCT_SUPPLIER.csv", encoding='utf-8')


In [3]:
data_delay = pd.concat([data_delay_4_6,data_delay_7_9],ignore_index=True)
data_not_delay = pd.concat([data_not_delay_4_6,data_not_delay_7_9],ignore_index=True)

full_data = pd.concat([data_delay,data_not_delay],ignore_index=True)
full_data = full_data.sample(frac = 1).drop_duplicates().reset_index()

full_data = full_data.drop('index',axis=1)

# EDA

In [4]:
full_data['OTHER AREA SHIP DIV'] = full_data['OTHER AREA SHIP DIV'].replace([' ', np.nan], '0')
pd.Series({col: full_data[col].unique() for col in full_data.columns})

Order date                         [2022-09-28, 2022-07-21, 2022-07-13, 2022-07-0...
SUBSIDIARY_CD                                                                  [MJP]
GLOBAL_NO                          [GCW00GGI002117, 09100696508514, UA93495500210...
CLASSIFY_CD                        [3933107, 3743528, 3632511, 3731226, 21038601,...
CUST_CD                            [251577, 100696, 62843, 5264, 532284, 726862, ...
BRAND_CD                           [MSM1, NTA1, KZC1, MDS1, TRC1, UNT1, PIS1, SUS...
INNER_CD                           [30123550003, 30485690001, 30390190004, 790085...
SUPPLIER_CD                        [0627, 3044, 3764, 3013, 9102, 0257, 3186, 014...
Sales order line number            [7, 1, 13, 38, 4, 320, 19, 3, 2, 18, 12, 15, 2...
Stock class                                                                   [1, 0]
Consider count hodiday Saturday                          [4, 0, 1, 4, 0, 1, 3, 3,  ]
SO QTY                             [1, 2, 4, 8, 5, 6, 10, 3, 24, 

In [5]:
features = full_data.columns.tolist()

removed_cols = ['QTUF_RCV_NO', 'SOUF_RCV_NO', 'WEIGHT_UNIT', 'PRODUCT_ASSORT',
                'SPECIFY_SHIP_DAYS', 'HEAVY_FLG', 'EXPENSIVE_FLG', 'ACTUAL_SHIP_DAYS',
                'SPECIFY_PRODUCTION_DAYS', 'HAZARD_FLG', 'IO_UNFIT_FLG',
                'SUPPLIER_CATEGORY_CD', 'REASON_CD', 'SUBSIDIARY_CD', 'GLOBAL_NO',
                'Stock class', 'SPECIAL DIV', 'PACK QTY', 'SO_TIME']

In [6]:
filtered_features = [feature for feature in features if feature not in removed_cols]

In [7]:
filter_data_delay = data_delay[filtered_features]
filter_data_not_delay = data_not_delay[filtered_features]

full_filter_data = pd.concat([filter_data_delay, filter_data_not_delay], ignore_index=True)
full_filter_data = full_filter_data.sample(frac = 1).drop_duplicates().reset_index()

full_filter_data = full_filter_data.drop('index',axis = 1)
filtered_features.remove('label')
test = test[filtered_features]


# BASIC EDA

In [9]:
full_filter_data['OTHER AREA SHIP DIV'] = full_filter_data['OTHER AREA SHIP DIV'].replace([' ', np.nan], '0')
full_filter_data['Consider count hodiday Saturday'] = full_filter_data['Consider count hodiday Saturday'].replace([' '], '0')

full_filter_data['VSD'] = pd.to_datetime(full_filter_data['VSD'], format='mixed')
full_filter_data['Order date'] = pd.to_datetime(full_filter_data['Order date'], format='mixed')
# full_filter_data['MONTH'] = full_filter_data['Order date'].dt.month
# full_filter_data['DAY'] = full_filter_data['Order date'].dt.day
# full_filter_data['YEAR'] = full_filter_data['Order date'].dt.year

full_filter_data['Consider count hodiday Saturday'] = full_filter_data['Consider count hodiday Saturday'].astype('int')
full_filter_data['OTHER AREA SHIP DIV'] = full_filter_data['OTHER AREA SHIP DIV'].astype('int64')

full_filter_data['WEIGHT'] = full_filter_data['WEIGHT PER PIECE'] * full_filter_data['SO QTY']
full_filter_data['day_range'] = full_filter_data['VSD'] - full_filter_data['Order date']
full_filter_data['day_range'] = full_filter_data['day_range'].astype('int64')

full_filter_data = full_filter_data.drop(['WEIGHT PER PIECE','SO QTY', 'VSD', 'Order date'],axis=1)

In [10]:
test['OTHER AREA SHIP DIV'] = test['OTHER AREA SHIP DIV'].replace([' ', np.nan], '0')
test['Consider count hodiday Saturday'] = test['Consider count hodiday Saturday'].replace([' '], '0')

test['VSD'] = pd.to_datetime(test['VSD'], format='mixed')
test['Order date'] = pd.to_datetime(test['Order date'], format='mixed')
# test['MONTH'] = test['Order date'].dt.month
# test['DAY'] = test['Order date'].dt.day
# test['YEAR'] = test['Order date'].dt.year

test['Consider count hodiday Saturday'] = test['Consider count hodiday Saturday'].astype('int')
test['OTHER AREA SHIP DIV'] = test['OTHER AREA SHIP DIV'].astype('int64')

test['WEIGHT'] = test['WEIGHT PER PIECE'] * test['SO QTY']
test['day_range'] = test['VSD'] - test['Order date']
test['day_range'] = test['day_range'].astype('int64')

test = test.drop(['WEIGHT PER PIECE','SO QTY', 'VSD', 'Order date'],axis=1)

In [ ]:
full_filter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470222 entries, 0 to 1470221
Data columns (total 26 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   CLASSIFY_CD                      1470222 non-null  int64  
 1   CUST_CD                          1470222 non-null  int64  
 2   BRAND_CD                         1470222 non-null  object 
 3   INNER_CD                         1470222 non-null  object 
 4   SUPPLIER_CD                      1470222 non-null  object 
 5   Sales order line number          1470222 non-null  int64  
 6   Consider count hodiday Saturday  1470222 non-null  int64  
 7   OTHER AREA SHIP DIV              1470222 non-null  int64  
 8   ALLOCATION QTY                   1470222 non-null  int64  
 9   SUPPLIER INV AMOUNT              1470222 non-null  float64
 10  PACKING RANK                     1470222 non-null  object 
 11  PRODUCT_CD                       1470222 non-null 

# More EDA

In [11]:
categorical_columns = [
    'CLASSIFY_CD', 'CUST_CD', 'BRAND_CD', 'INNER_CD', 'SUPPLIER_CD',
    'OTHER AREA SHIP DIV', 'PACKING RANK', 'PRODUCT_CD',
    'PRODUCT ATTRIBUTION', 'Ship Mode', 'DELI_DIV'
    
]

numerical_columns = ['day_range', 'WEIGHT', 'SUPPLIER INV AMOUNT']

In [12]:
for col in categorical_columns:
    full_filter_data[col] = full_filter_data[col].astype('category')

for col in categorical_columns:
    test[col] = test[col].astype('category')

In [ ]:
full_filter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470222 entries, 0 to 1470221
Data columns (total 26 columns):
 #   Column                           Non-Null Count    Dtype   
---  ------                           --------------    -----   
 0   CLASSIFY_CD                      1470222 non-null  category
 1   CUST_CD                          1470222 non-null  category
 2   BRAND_CD                         1470222 non-null  category
 3   INNER_CD                         1470222 non-null  category
 4   SUPPLIER_CD                      1470222 non-null  category
 5   Sales order line number          1470222 non-null  int64   
 6   Consider count hodiday Saturday  1470222 non-null  int64   
 7   OTHER AREA SHIP DIV              1470222 non-null  category
 8   ALLOCATION QTY                   1470222 non-null  int64   
 9   SUPPLIER INV AMOUNT              1470222 non-null  float64 
 10  PACKING RANK                     1470222 non-null  category
 11  PRODUCT_CD                       1470

In [13]:
X = full_filter_data.drop(['label'], axis=1)
y = full_filter_data['label']

In [14]:
X_categorical = X[categorical_columns]
X_numerical = X[numerical_columns]
X_other = X.drop(categorical_columns + numerical_columns, axis=1)

test_categorical = test[categorical_columns]
test_numerical = test[numerical_columns]
test_other = test.drop(categorical_columns + numerical_columns, axis=1)

In [15]:
X_other = X_other.drop(['PURCHASE AMOUNT'], axis=1)
test_other = test_other.drop(['PURCHASE AMOUNT'], axis=1)

In [16]:
from category_encoders import CountEncoder

# Encode categorical features using CountEncoder
count_encoder = CountEncoder()
X_categorical = count_encoder.fit_transform(X_categorical)
test_categorical = count_encoder.transform(test_categorical)

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\ordinal.py:198: FutureWa

In [17]:
from sklearn.preprocessing import StandardScaler

# Initialize StandardScaler
scaler = StandardScaler()

# Scale input features
X_scaled = scaler.fit_transform(X_numerical)
test_scaled = scaler.transform(test_numerical)

In [18]:
X_scaled = pd.DataFrame(X_scaled, columns=X_numerical.columns)
X = pd.concat([X_other, X_scaled, X_categorical], axis=1)

test_scaled = pd.DataFrame(test_scaled, columns=test_numerical.columns)
test = pd.concat([test_other, test_scaled, test_categorical], axis=1)

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.inspection import permutation_importance
# from lightgbm import LGBMClassifier

# # Construct numerical data

# # Split into train and test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Create model
# model = LGBMClassifier()

# # Fit to data
# model.fit(X_train, y_train)

# # Loop for calculate importance of each features
# result = permutation_importance(
#     model, X_test, y_test, n_repeats=10, random_state=42
# )

# # Display result
# print(pd.DataFrame(
#     {k: result[k] for k in ("importances_mean", "importances_std",)},
#     index=X_test.columns
# ).sort_values(by="importances_mean", ascending=False))

In [19]:
X = X.drop('SHIP DECISION NO', axis=1)
test = test.drop('SHIP DECISION NO', axis=1)

In [20]:
X = X.drop('PRODUCT ATTRIBUTION', axis=1)
test = test.drop('PRODUCT ATTRIBUTION', axis=1)

In [21]:
X = X.drop('OTHER AREA SHIP DIV', axis=1)
test = test.drop('OTHER AREA SHIP DIV', axis=1)

In [ ]:
# X = X.drop('Sales order line number', axis=1)
# test = test.drop('Sales order line number', axis=1)

In [ ]:
# X = X.drop('ALLOCATION QTY', axis=1)
# test = test.drop('ALLOCATION QTY', axis=1)

# Model test

In [ ]:
# from sklearn.metrics import classification_report
# from lightgbm import LGBMClassifier

# model = LGBMClassifier()
# model.fit(X, y)

# pred = model.predict(test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 35538, number of negative: 1434684
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.230961 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2597
[LightGBM] [Info] Number of data points in the train set: 1470222, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.024172 -> initscore=-3.698097
[LightGBM] [Info] Start training from score -3.698097


In [24]:
from sklearn.metrics import classification_report
# from xgboost import XGBClassifier
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    # iterations=500,  # Giảm số lượng iterations
    learning_rate=0.1,
    eval_metric='AUC',
    # cat_features=categorical_features,
    random_seed=42,
    # use_best_model=True
)
model.fit(X, y)

pred = model.predict(test)

0:	total: 415ms	remaining: 6m 54s
1:	total: 633ms	remaining: 5m 16s
2:	total: 885ms	remaining: 4m 54s
3:	total: 1.13s	remaining: 4m 41s
4:	total: 1.38s	remaining: 4m 34s
5:	total: 1.61s	remaining: 4m 27s
6:	total: 1.85s	remaining: 4m 23s
7:	total: 2.12s	remaining: 4m 22s
8:	total: 2.43s	remaining: 4m 27s
9:	total: 2.86s	remaining: 4m 42s
10:	total: 3.08s	remaining: 4m 36s
11:	total: 3.35s	remaining: 4m 35s
12:	total: 3.58s	remaining: 4m 31s
13:	total: 3.8s	remaining: 4m 27s
14:	total: 4.1s	remaining: 4m 29s
15:	total: 4.41s	remaining: 4m 31s
16:	total: 4.71s	remaining: 4m 32s
17:	total: 5s	remaining: 4m 32s
18:	total: 5.37s	remaining: 4m 37s
19:	total: 5.69s	remaining: 4m 38s
20:	total: 5.93s	remaining: 4m 36s
21:	total: 6.26s	remaining: 4m 38s
22:	total: 6.47s	remaining: 4m 34s
23:	total: 6.73s	remaining: 4m 33s
24:	total: 6.96s	remaining: 4m 31s
25:	total: 7.22s	remaining: 4m 30s
26:	total: 7.47s	remaining: 4m 29s
27:	total: 7.68s	remaining: 4m 26s
28:	total: 7.92s	remaining: 4m 25s


In [26]:
from collections import Counter
Counter(pred)

Counter({0: 122960, 1: 2556})

In [28]:
result = pd.DataFrame({
    'label':pred},index=[i for i in range(1,125517)])
result.to_csv(r'D:\Code\Python\TH_Tien_Xu_Ly\ds-108-o-21-assigment-06\prediction (13).csv')

In [29]:
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, auc, f1_score, confusion_matrix, recall_score, classification_report
import seaborn as sns


# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')

def display_roc_curve(y_, oof_preds_,sub_preds_,folds_idx_):
    # Plot ROC curves
    plt.figure(figsize=(6,6))
    scores = []
    for n_fold, (_, val_idx) in enumerate(folds_idx_):
        # Plot the roc curve
        fpr, tpr, thresholds = roc_curve(y_.iloc[val_idx], oof_preds_[val_idx])
        score = 2 * auc(fpr, tpr) -1
        scores.append(score)
        plt.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC fold %d (Gini = %0.4f)' % (n_fold + 1, score))

    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)
    fpr, tpr, thresholds = roc_curve(y_, oof_preds_)
    score = 2 * auc(fpr, tpr) -1
    plt.plot(fpr, tpr, color='b',
             label='Avg ROC (Gini = %0.4f $\pm$ %0.4f)' % (score, np.std(scores)),
             lw=2, alpha=.8)

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('LightGBM ROC Curve')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig('roc_curve.png')


# LightGBM GBDT with Stratified KFold
def kfold_lightgbm(train_df, test_df, num_folds, stratified = False, debug= False):
    # Divide in training/validation and test data
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    gc.collect()
    # Cross validation model
    folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=500)

    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['label','id']]

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['label'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['label'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['label'].iloc[valid_idx]

        clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=128,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1
        )

        clf.fit(train_x, train_y.ravel(), eval_set=[(train_x, train_y), (valid_x, valid_y)],
            eval_metric='auc', verbose= 1000, early_stopping_rounds= 200)

        oof_pred = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]

        pred = clf.predict(valid_x, num_iteration=clf.best_iteration_)
        print('F1 Score: ' + str( f1_score(valid_y, pred) ))
        print('Recall Score: ' + str( recall_score(valid_y, pred) ))

        sub_pred = clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
        oof_preds[valid_idx] = oof_pred
        sub_preds += sub_pred

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['label'], oof_preds))

    folds_idx = [(trn_idx, val_idx) for trn_idx, val_idx in folds.split(train_df[feats], train_df['label'])]
    #display_roc_curve(y_=train_df['label'],oof_preds_=oof_preds,sub_preds_ = sub_preds, folds_idx_=folds_idx)

    # Write submission file and plot feature importance
    display_importances(feature_importance_df.tail(30))

In [ ]:
kfold_lightgbm(X, y, 5)

In [ ]:
# sample = pd.read_csv('/content/drive/MyDrive/Tiền xử lí và xây dựng bộ dữ liệu/Baidiemcong_TH/sample_Solution.csv')

In [ ]:
# prediction = pd.DataFrame(pred, columns = ['label'])

# sample = sample.drop('label', axis=1)
# prediction = pd.concat([sample, prediction], axis=1)